In [3]:
!pip freeze > requirements.txt

In [1]:
# === Импорт необходимых библиотек ===
import ee
import geemap
import rasterio
import torch
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
from pyproj import Geod
from rasterio.mask import mask
import torchvision.transforms as T
from torchvision.models.segmentation import deeplabv3_resnet50
import matplotlib.pyplot as plt
import cv2

KeyboardInterrupt: 

In [ ]:
# === 1. Инициализация Earth Engine ===
ee.Initialize()

In [19]:
# === 2. Общие вспомогательные функции ===
def select_region(lon_min, lat_min, lon_max, lat_max):
    """Создает область интереса на основе координат."""
    return ee.Geometry.Rectangle([lon_min, lat_min, lon_max, lat_max])

def get_coordinates():
    """Запрашивает у пользователя координаты прямоугольника."""
    try:
        print("Введите координаты прямоугольника (долгота и широта):")
        lon_min = float(input("Минимальная долгота (lon_min): "))
        lat_min = float(input("Минимальная широта (lat_min): "))
        lon_max = float(input("Максимальная долгота (lon_max): "))
        lat_max = float(input("Максимальная широта (lat_max): "))
        return lon_min, lat_min, lon_max, lat_max
    except ValueError:
        print("Ошибка: Введите корректные числа для координат.")
        return get_coordinates()

def calculate_area_from_coordinates(lon_min, lat_min, lon_max, lat_max):
    """Рассчитывает площадь прямоугольного участка в квадратных метрах."""
    rectangle = box(lon_min, lat_min, lon_max, lat_max)
    geod = Geod(ellps="WGS84")
    area, _ = geod.geometry_area_perimeter(rectangle)
    return abs(area)

In [ ]:
# === 3. Работа с данными качества воздуха ===
def fetch_air_quality_data(region, start_date='2024-06-01', end_date='2024-06-15'):
    """Получает данные по NO2, CO, CH4 и SO2."""
    no2 = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_NO2").filterDate(start_date, end_date) \
        .select('tropospheric_NO2_column_number_density').mean().clip(region)
    co = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_CO").filterDate(start_date, end_date) \
        .select('CO_column_number_density').mean().clip(region)
    ch4 = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CH4").filterDate(start_date, end_date) \
        .select('CH4_column_volume_mixing_ratio_dry_air').mean().clip(region)
    so2 = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_SO2").filterDate(start_date, end_date) \
        .select('SO2_column_number_density').mean().clip(region)
    return no2, co, ch4, so2

def export_data_to_geotiff(data, filename, region):
    """Экспортирует данные в GeoTIFF."""
    geemap.ee_export_image(data, filename=filename, scale=1000, region=region.getInfo())

def calculate_average_pollution(region, files):
    """Вычисляет средние значения загрязнителей по GeoTIFF-файлам."""
    averages = {}
    for pollutant, file_path in files.items():
        with rasterio.open(file_path) as src:
            out_image, _ = mask(src, [region], crop=True)
            values = out_image[0]
            values = values[values > 0]
            averages[pollutant] = np.mean(values) if values.size > 0 else 0
    return averages

In [24]:
# === 4. Работа с изображениями и сегментацией ===
def segment_image(image_path, model, label_colors):
    """Выполняет сегментацию изображения."""
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    transform = T.Compose([
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    input_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        output = model(input_tensor)['out'][0]
    output_predictions = torch.argmax(output, dim=0).cpu().numpy()
    segmented_image = label_colors[output_predictions]
    return img, segmented_image

def visualize_segmentation(original, segmented):
    """Отображает оригинальное изображение и результат сегментации."""
    fig, axes = plt.subplots(1, 2, figsize=(15, 8))
    axes[0].imshow(original)
    axes[0].set_title("Оригинальное изображение")
    axes[0].axis("off")
    axes[1].imshow(segmented.astype('uint8'))
    axes[1].set_title("Сегментированное изображение")
    axes[1].axis("off")
    plt.show()

In [ ]:
# === 5. Работа с детекцией объектов ===
def detect_objects(image_path, model_path, img_size=640, conf_thresh=0.5):
    """Детекция объектов на изображении с использованием YOLOv5."""
    try:
        model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
        model.conf = conf_thresh
        model.imgsz = img_size
        results = model(image_path)
        return results.pandas().xyxy[0]
    except Exception as e:
        print(f"Ошибка при загрузке модели YOLOv5: {e}")
        return pd.DataFrame()

In [ ]:
# === Основной код ===
# Ввод координат от пользователя
lon_min, lat_min, lon_max, lat_max = get_coordinates()
region = select_region(lon_min, lat_min, lon_max, lat_max)
region_geometry = box(lon_min, lat_min, lon_max, lat_max)
total_area = calculate_area_from_coordinates(lon_min, lat_min, lon_max, lat_max)
print(f"Площадь участка: {total_area:.2f} м²")

In [ ]:
# Загрузка данных о качестве воздуха
no2_data, co_data, ch4_data, so2_data = fetch_air_quality_data(region)
export_data_to_geotiff(no2_data, "no2_concentration.tif", region)
export_data_to_geotiff(co_data, "co_concentration.tif", region)
export_data_to_geotiff(ch4_data, "ch4_concentration.tif", region)
export_data_to_geotiff(so2_data, "so2_concentration.tif", region)
print("Данные о качестве воздуха успешно экспортированы в GeoTIFF.")

In [ ]:
# Путь к модели YOLOv5 и изображению
model_path = "car_detection/yolov5/runs/train/weights/best.pt"
image_path = "path/to/your/image.jpg"

# Запуск детекции транспорта
detections = detect_objects(image_path, model_path)

# Подсчёт транспортных средств
transport_data = {
    'cars': len(detections[detections['class'] == 0]),
    'buses': len(detections[detections['class'] == 1]),
    'trucks': len(detections[detections['class'] == 2])
}
print(f"Данные о транспорте: {transport_data}")
for vehicle, count in transport_data.items():
    print(f"- {vehicle}: {count}")

In [2]:
# Инициализация модели сегментации
model = deeplabv3_resnet50(pretrained=True)
model.eval()
LABEL_COLORS = np.array([
    (0, 0, 0),       # Background
    (128, 0, 0),     # Buildings
    (0, 128, 0),     # Parks/Forest
    (128, 128, 0),   # Roads
    (0, 0, 128),     # Water
    (128, 0, 128)    # Other
])

NameError: name 'deeplabv3_resnet50' is not defined

In [ ]:
# Выполнение сегментации изображения
original, segmented = segment_image(image_path, model, LABEL_COLORS)

# Визуализация результатов сегментации
visualize_segmentation(original, segmented)

In [ ]:
# Коэффициенты влияния земельного использования на загрязнители
land_use = {'roads': 0.15, 'buildings': 0.25, 'parks': 0.20, 'water': 0.10}
updated_land_use = land_use.copy()

# Ввод параметров строительства
new_building_area = float(input("Площадь нового здания (м²): "))
new_roads_area = float(input("Площадь новых дорог (м²): "))
updated_land_use['roads'] += new_roads_area / total_area
updated_land_use['buildings'] += new_building_area / total_area
updated_land_use['parks'] -= (new_building_area + new_roads_area) / total_area
updated_land_use['parks'] = max(updated_land_use['parks'], 0)

In [ ]:
# Рассчитываем изменения загрязнителей
pollution_files = {
    'NO2': "no2_concentration.tif",
    'CO': "co_concentration.tif",
    'CH4': "ch4_concentration.tif",
    'SO2': "so2_concentration.tif"
}
current_pollution = calculate_average_pollution(region_geometry, pollution_files)

print("\nСредние значения загрязнителей воздуха на участке:")

In [ ]:
for pollutant, value in current_pollution.items():
    print(f"- {pollutant}: {value:.6f}")

pollution_change = {}
for pollutant in current_pollution.keys():
    land_use_change = sum(
        (updated_land_use[land_type] - land_use[land_type]) * land_use_impact_factors[pollutant].get(land_type, 0) * total_area
        for land_type in land_use
    )
    transport_change = sum(
        transport_impact_factors[pollutant][vehicle_type] * count
        for vehicle_type, count in transport_data.items()
    )
    pollution_change[pollutant] = land_use_change + transport_change

new_pollution = {pollutant: current_pollution[pollutant] + pollution_change[pollutant] 
                 for pollutant in current_pollution}

print("\n=== Рекомендации для строительной компании ===")
print(f"Площадь участка: {total_area:.2f} м²")
print(f"Площадь нового здания: {new_building_area:.2f} м²")
print(f"Площадь новых дорог: {new_roads_area:.2f} м²")

for pollutant, change in pollution_change.items():
    print(f"\nОжидаемое изменение {pollutant}: {change:+.6f} mol/m²")
    if change > 0:
        print(f"Рекомендация: Рассмотрите компенсационные меры для снижения {pollutant}.")
    else:
        print(f"{pollutant} снизится, что благоприятно для окружающей среды.")

if updated_land_use['parks'] < 0.1:
    print("\nПарковая зона существенно сократилась. Рекомендуется высадить деревья или сохранить оставшуюся зелень.")